In [17]:
from abc import ABCMeta, abstractmethod
from pathlib import Path
from uuid import uuid4
from typing import Optional, Callable, List
import inspect

import panel as pn
from panel.viewable import Viewer

import param
from param import rx, Parameterized

from langchain_core.language_models import BaseLanguageModel
from langchain_openai import ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_chroma import Chroma # Useless
from langchain_core.documents import Document
from langchain_core.retrievers import BaseRetriever
from langchain_core.documents import Document

import chromadb
from chromadb.api.client import Client
from chromadb.api.models.Collection import Collection

import tiktoken

from dotenv import load_dotenv


load_dotenv()
pn.extension()

# Interface Explore

In [34]:
spacer = pn.Spacer(
    height=100,
    styles={"background": "teal"},
)
spacer0 = pn.Spacer(
    height=100,
    styles={"background": "red"},
)

In [35]:
gspec = pn.GridSpec(width=800, height=600)

In [36]:
gspec[0, :] = spacer

In [39]:
gspec[1, :] = spacer0


    (1, 0): Spacer(height=300, styles={'background': 'teal'}, width=800)

The following shows a view of the grid (empty: 0, occupied: 1, overlapping: 2):

[[1]
 [2]]


In [45]:
pn.Column(spacer, pn.Spacer(height=20), spacer0)

BokehModel(combine_events=True, render_bundle={'docs_json': {'e01e7589-7171-4850-87ae-6e5254501dfc': {'version…

In [90]:
clear_chat_button = pn.widgets.Button(name="Clear Chat", width=100)
model_chooser = pn.widgets.Select(name="LLM", options=["Model 1", "Model 2", "Model 3"])
corpus_chooser = pn.widgets.Select(
    name="Corpus", options=["Corpus 1", "Corpus 2", "Corpus 3"]
)

chat_sidebar = pn.Column(
    clear_chat_button, corpus_chooser, model_chooser, align="center"
)

In [91]:
chat_feed = pn.chat.ChatFeed(height=400, width=800)

chat_input = pn.chat.ChatAreaInput(width=600, rows=3)
send_button = pn.widgets.Button(name="Send", width=140, height=60)
toggle_retriever = pn.widgets.Toggle(name="Retriever", value=False, width=60, height=60)
chat_send_row = pn.Row(chat_input, send_button, toggle_retriever)

main_chat = pn.Column(chat_feed, chat_send_row)

In [92]:
chat_tab = pn.Row(sidebar, main_chat)
chat_tab

BokehModel(combine_events=True, render_bundle={'docs_json': {'20a9b3ea-bc99-49aa-99ee-3c9d69449943': {'version…

In [106]:
new_corpus_button = pn.widgets.Button(name="New Corpus", width=100)
corpus_name_box = pn.widgets.TextInput(name="Corpus Name", width=200)

strategy_select = pn.widgets.Select(
    name="Strategy", options=["Strategy 1", "Strategy 2", "Strategy 3"]
)
parameters = pn.Column("## Parameters", styles={"background": "whitesmoke"})

corpus_left = pn.Column(new_corpus_button, corpus_name_box, strategy_select, parameters)
corpus_left

BokehModel(combine_events=True, render_bundle={'docs_json': {'e21f5372-801d-449c-b89a-d365f6e00af6': {'version…

In [103]:
current_corpus_name = pn.pane.Markdown(object="# Current Corpus: None")

file_input = pn.widgets.FileInput(name="Upload File")
file_input_button = pn.widgets.Button(name="Upload File", width=100)
file_input_row = pn.Row(file_input, file_input_button)

site_loader_input = pn.widgets.TextInput(
    name="Site Loader", value="https://www.example.com"
)
site_loader_button = pn.widgets.Button(name="Load Site", width=100)
site_loader_row = pn.Row(site_loader_input, site_loader_button)


corpus_mid = pn.Column(
    current_corpus_name, corpus_chooser, file_input_row, site_loader_row
)
corpus_mid

BokehModel(combine_events=True, render_bundle={'docs_json': {'681b277e-fb44-42a9-b162-b028bf324e3d': {'version…

In [105]:
resource_list_to_process = pn.Column(
    "# Resources to Process", styles={"background": "whitesmoke"}
)
resource_list_process = pn.widgets.Button(name="Process", width=100)

resource_list = pn.Column("# Resources Present", styles={"background": "whitesmoke"})

resource_progress = pn.pane.Markdown(object="## Progress")

corpus_right = pn.Column(
    resource_list_to_process, resource_list_process, resource_list, resource_progress
)
corpus_right

BokehModel(combine_events=True, render_bundle={'docs_json': {'6427310c-e1b1-4253-a325-6dbf07207814': {'version…

In [107]:
corpus_tab = pn.Row(corpus_left, corpus_mid, corpus_right)
corpus_tab

BokehModel(combine_events=True, render_bundle={'docs_json': {'5eb51c56-81c7-452b-bf7d-75dffd9d9e54': {'version…

In [116]:
srv.stop()

AssertionError: Already stopped

In [135]:
pn.config.theme = "default"

In [133]:
serve(
    pn.Tabs(
        ("# Chat", chat_tab),
        ("Corpus Settings", corpus_tab),
        stylesheets=[".bk-tab {font-size: 20px}"],
    )
)

Launching server at http://localhost:35594


In [42]:
srv = None


def serve(obj):
    global srv
    try:
        srv.stop()
    except:
        srv = pn.serve(obj, port=35594)
        return srv

# Business Logic Class Construction

## Chat model

In [ ]:
chat_model = ChatOpenAI(model_name="gpt-3.5-turbo")
chat_model

In [56]:
chat_model = ChatOpenAI()
# To get the arguments of the chat model, you can use the 'inspect' module
import inspect

print(inspect.getfullargspec(chat_model.__init__))

FullArgSpec(args=['__pydantic_self__'], varargs=None, varkw='data', defaults=None, kwonlyargs=[], kwonlydefaults=None, annotations={'return': None, 'data': typing.Any})


In [372]:
class LLMchatModel(param.Parameterized):
    model_class = param.ClassSelector(
        class_=BaseLanguageModel,
        is_instance=False,
        doc='Class of a LangChain Chat Model')
    model_args = param.ClassSelector(
        class_=(dict, list),
        doc='''Takes a dictionary of arguments to pass to expose and send to the
        model class. If you set a None value as the key, the argument will be exposed,
        with the default value set. Passing a list is the same as passing a dict
        with all values set to None.''',
        is_instance=True)

    model = param.ClassSelector(class_=BaseLanguageModel, is_instance=True)

    def __init__(self, **kwargs):
        super().__init__(**kwargs)

        self._set_params()
        self._create_model()

    def _set_params(self):
        """Handles setting the params on init"""
        if self.model_args:
            for arg, val in self.model_args.items():
                if arg in self.model_class.__fields__:
                    if val is None:
                        default = self.model_class.__fields__[arg].default
                        self.model_args[arg] = default
                        self.param.add_parameter(arg, param.Parameter(default, per_instance=True))
                    else:
                        self.param.add_parameter(arg, param.Parameter(val, per_instance=True))
                    # self.model_args_list.append(arg)
                else:
                    raise ValueError(f"'{arg}' is missing from the model class's signature")
                self.param.watch(self._create_model, [*self.model_args.keys()])


    def _create_model(self, event=None):
        """Creates the model instance on init and when any of the parameters change"""
        arg_vals = {arg: self.param.values()[arg] for arg in self.model_args.keys()}
        self.model = self.model_class(**arg_vals)

In [255]:
# class ChatModel(param.Parameterized):
#     model_class = param.ClassSelector(
#         class_=BaseLanguageModel,
#         is_instance=False,
#         doc='Class of a LangChain Chat Model')
#     model_args = param.ClassSelector(
#         class_=(dict, list),
#         doc='''Takes a dictionary of arguments to pass to expose and send to the
#         model class. If you set a None value as the key, the argument will be exposed,
#         with the default value set. Passing a list is the same as passing a dict
#         with all values set to None.''',
#         is_instance=True)

#     # model = param.ClassSelector(class_=BaseLanguageModel, is_instance=True)
#     model_args_list = param.List(default=[])

#     def __init__(self, **kwargs):
#         super().__init__(**kwargs)

#         self._set_params()
#         print('creating model')
#         self.model = rx(self._create_model)(
#             **{arg: getattr(self, arg) for arg in self.model_args_list}
#             )

#     def _set_params(self):
#         """Handles setting the params on init"""
#         if self.model_args:
#             for arg, val in self.model_args.items():
#                 if arg in self.model_class.__fields__:
#                     if val is None:
#                         default = self.model_class.__fields__[arg].default
#                         setattr(self, arg, rx(default))
#                     else:
#                         setattr(self, arg, rx(val))
#                     self.model_args_list.append(arg)
#                 else:
#                     raise ValueError(f"'{arg}' is missing from the model class's signature")
#                 # self.param.watch(self._create_model, [*self.model_args_list])

#     def _create_model(self, **kwargs):
#         """Creates the model instance on init and when any of the parameters change"""
#         # arg_vals = {arg: getattr(self, arg).rx.value for arg in self.model_args_list}
#         print(kwargs)
#         self.model = self.model_class(**kwargs)

In [373]:
c = ChatModel(model_class=ChatOpenAI, model_args={'model_name': 'gpt4'})
c

ChatModel(model=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7f9371244290>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f93707c72f0>, model_name='gpt4', openai_api_key=SecretStr('**********'), openai_proxy=''), model_args={'model_name': 'gpt4'}, model_class=<class 'langchain_openai.chat_models.base.ChatOpenAI'>, model_name='gpt4', name='ChatModel00804')

In [374]:
c.model_name = 'gpt35'

In [411]:
def set_model_name(self, value):
    self.model_name = value

widge = pn.widgets.Select(options=['gpt35', 'gpt4', 'sike'], value=c.param.model_name)
widge

BokehModel(combine_events=True, render_bundle={'docs_json': {'62c83a28-ed22-4611-bea5-ca57d8081fb2': {'version…

In [425]:
c.model_name

'gpt35'

In [421]:
class TestClass(param.Parameterized):
    some_param = param.String(default='test_value')

test_class = TestClass()

widget = pn.widgets.TextInput(value=test_class.param.some_param)

# def update_test_class_param(event):
#     test_class.some_param = event.new

# widget.param.watch(update_test_class_param, 'value')

widget

BokehModel(combine_events=True, render_bundle={'docs_json': {'00946c09-af8d-41e0-a856-1fa0265d6559': {'version…

In [424]:
test_class.some_param

'whaaat'

In [423]:
test_class.some_param = 'whaaat'

In [410]:
def update_test_class_param(val):
    test_class.some_param = val

widget.param.watch(update_test_class_param, 'value')


Watcher(inst=TextInput(value='whaaatdff', value_input='whaaatdff'), cls=<class 'panel.widgets.input.TextInput'>, fn=<function update_test_class_param at 0x7f93704e58a0>, mode='args', onlychanged=True, parameter_names=('value',), what='value', queued=False, precedence=0)

In [377]:

class Task(pn.viewable.Viewer):
    """A model of a Task"""

    value: str = param.String(doc="A description of the task")
    completed: bool = param.Boolean(
        doc="If True the task has been completed. Otherwise not."
    )

    def __panel__(self):
        completed = pn.widgets.Checkbox.from_param(
            self.param.completed, name="", align="center", sizing_mode="fixed"
        )
        content = pn.pane.Markdown(object=self.param.value)
        return pn.Row(completed, content, sizing_mode="stretch_width")

In [382]:
tsk = Task(value='wasssssssup')
pn.panel(tsk)


BokehModel(combine_events=True, render_bundle={'docs_json': {'1456147a-8f78-48fe-9b60-7f398557a0c6': {'version…

In [385]:
tsk.completed

False

In [383]:
tsk.completed = True

## Corpus

In [ ]:
class Corpus(param.Parameterized):
    name = param.String()
    corpus_creation_args = param.Dict(default={})
    corpus_retrieval_args = param.Dict(default={})
    
    # Creation logic
    # Retrieval logic
    # Storage logic - is it persistent? option?
    

## RAG

In [112]:
class ParameterizedABCMeta(ABCMeta, type(Parameterized)):
    pass

class BaseParameterized(Parameterized, metaclass=ParameterizedABCMeta):
    pass

class BaseRAG(BaseParameterized):
    # add params as necessary for external use
    @abstractmethod
    def create_corpus(self):
        pass

class SimpleRAG(BaseRAG):

    def create_corpus(self):
        pass

## Memory

# Skeleton Pipeline

## Load and split

In [2]:
def token_len(text, model_name='gpt-3.5-turbo'):
    encoding = tiktoken.encoding_for_model(model_name)
    encoding_len = len(encoding.encode(text))
    return encoding_len

In [3]:
tpdf_path = Path('examples/data/2112.01488v3.pdf')
tpdf_loader = UnstructuredPDFLoader(tpdf_path)
tpdf_doc = tpdf_loader.load()

tpdf_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50, length_function=token_len)
tpdf_chunks = tpdf_splitter.split_documents(tpdf_doc)
tpdf_chunks

[Document(page_content='2 2 0 2\n\nl u J\n\n0 1\n\n]\n\nR\n\nI . s c [\n\n3 v 8 8 4 1 0 . 2 1 1 2 : v i X r a\n\nColBERTv2: Effective and Efﬁcient Retrieval via Lightweight Late Interaction\n\nKeshav Santhanam∗ Stanford University\n\nOmar Khattab∗ Stanford University\n\nJon Saad-Falcon Georgia Institute of Technology\n\nChristopher Potts Stanford University\n\nMatei Zaharia Stanford University\n\nAbstract\n\nNeural information retrieval (IR) has greatly advanced knowledge- and intensive language tasks. While many neural IR methods encode queries and documents into late interaction models produce multi-vector repre- sentations at the granularity of each token and decompose relevance modeling into scalable token-level computations. This decomposition has been shown to make late interaction more effective, but it inﬂates the space footprint of these models by an order of magnitude. In this work, we introduce ColBERTv2, a retriever that couples an aggressive residual compres- sion mechanis

## Encoding

In [4]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

# For a retrieval task that uses short queries to find long related documents,
# it is recommended to add instructions for these short queries. 
emb_model_name = 'BAAI/bge-large-en-v1.5' 
emb_model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceBgeEmbeddings(
    model_name=emb_model_name, model_kwargs=emb_model_kwargs, encode_kwargs=encode_kwargs
)
hf


/root/miniconda3/envs/RAGular/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


HuggingFaceBgeEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='BAAI/bge-large-en-v1.5', cache_folder=None, model_kwargs={'device': 'cpu'}, encode_kwargs={'normalize_embeddings': True}, query_instruction='Represent this question for searching relevant passages: ', embed_instruction='')

In [5]:
embedded_chunks = hf.embed_documents([_.page_content for _ in tpdf_chunks]) # Takes a list of strings instead of documents(???)
embedded_chunks

[[0.027144655585289,
  0.010908291675150394,
  -0.007803196553140879,
  0.010273820720613003,
  -0.0004490670107770711,
  -0.005330324172973633,
  -0.03391449153423309,
  -0.04463495314121246,
  -0.01265870314091444,
  0.04743312671780586,
  0.0319766141474247,
  -0.012839773669838905,
  -0.03520369529724121,
  -0.04842870309948921,
  -0.017946913838386536,
  -0.05331189185380936,
  -0.011299281381070614,
  -0.025722434744238853,
  -0.04562721028923988,
  -0.03869546577334404,
  0.028745662420988083,
  0.01883101277053356,
  -0.06315965950489044,
  0.010715173557400703,
  -0.0074507747776806355,
  0.010830319486558437,
  0.02072223089635372,
  0.02249465137720108,
  0.08006232976913452,
  0.04530643671751022,
  0.0470992811024189,
  -0.05207795649766922,
  -0.0023492886684834957,
  -0.017810437828302383,
  -0.00837847962975502,
  -0.06615505367517471,
  0.03105706349015236,
  -0.005657607689499855,
  0.012417525053024292,
  -0.036523349583148956,
  0.0315505750477314,
  -0.033478498458

## Vector Store


In [430]:
# Corpus folder creation
corpus_local = Path('corpus_local')
simple_rag_path = corpus_local / 'simple_rag'
simple_rag_path.mkdir(parents=True, exist_ok=True)

In [ ]:
chroma = Chroma(collection_name='simple_rag0', embedding_function)



In [7]:
# TODO: create new client each time or have a pool of clients?
client = chromadb.PersistentClient(path=str(simple_rag_path / 'chromadb'))

In [62]:
collection = client.get_or_create_collection()


In [67]:
hf.embed_documents

<bound method HuggingFaceBgeEmbeddings.embed_documents of HuggingFaceBgeEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='BAAI/bge-large-en-v1.5', cache_folder=None, model_kwargs={'device': 'cpu'}, encode_kwargs={'normalize_embeddings': True}, query_instruction='Represent this question for searching relevant passages: ', embed_instruction='')>

In [8]:
collection = client.get_or_create_collection(name='simple_rag')

In [80]:
tpdf_chunks

[Document(page_content='2 2 0 2\n\nl u J\n\n0 1\n\n]\n\nR\n\nI . s c [\n\n3 v 8 8 4 1 0 . 2 1 1 2 : v i X r a\n\nColBERTv2: Effective and Efﬁcient Retrieval via Lightweight Late Interaction\n\nKeshav Santhanam∗ Stanford University\n\nOmar Khattab∗ Stanford University\n\nJon Saad-Falcon Georgia Institute of Technology\n\nChristopher Potts Stanford University\n\nMatei Zaharia Stanford University\n\nAbstract\n\nNeural information retrieval (IR) has greatly advanced knowledge- and intensive language tasks. While many neural IR methods encode queries and documents into late interaction models produce multi-vector repre- sentations at the granularity of each token and decompose relevance modeling into scalable token-level computations. This decomposition has been shown to make late interaction more effective, but it inﬂates the space footprint of these models by an order of magnitude. In this work, we introduce ColBERTv2, a retriever that couples an aggressive residual compres- sion mechanis

In [12]:
def kv_to_str(input_dict):
    return {str(k): str(v) for k, v in input_dict.items()}

metadata = kv_to_str(tpdf_chunks[0].metadata)
metadata


{'source': 'examples/data/2112.01488v3.pdf'}

In [14]:
collection.add(ids=[str(uuid4()) for _ in embedded_chunks],
               embeddings=embedded_chunks,
               documents=[_.page_content for _ in tpdf_chunks],
               metadatas=[kv_to_str(_.metadata) for _ in tpdf_chunks])

In [70]:
client.delete_collection('simple_rag')

In [23]:
type(client)

chromadb.api.client.Client

In [49]:
tpdf_chunks[0]

Document(page_content='2 2 0 2\n\nl u J\n\n0 1\n\n]\n\nR\n\nI . s c [\n\n3 v 8 8 4 1 0 . 2 1 1 2 : v i X r a\n\nColBERTv2: Effective and Efﬁcient Retrieval via Lightweight Late Interaction\n\nKeshav Santhanam∗ Stanford University\n\nOmar Khattab∗ Stanford University\n\nJon Saad-Falcon Georgia Institute of Technology\n\nChristopher Potts Stanford University\n\nMatei Zaharia Stanford University\n\nAbstract\n\nNeural information retrieval (IR) has greatly advanced knowledge- and intensive language tasks. While many neural IR methods encode queries and documents into late interaction models produce multi-vector repre- sentations at the granularity of each token and decompose relevance modeling into scalable token-level computations. This decomposition has been shown to make late interaction more effective, but it inﬂates the space footprint of these models by an order of magnitude. In this work, we introduce ColBERTv2, a retriever that couples an aggressive residual compres- sion mechanism

In [28]:
results = collection.query(query_embeddings=hf.embed_documents(['What is the capital of France?']))
results


{'ids': [['720959dc-4f12-4ad5-841b-f54403baf9d4',
   'dca00914-63f4-44c7-9ada-7376f20dcdf0',
   'a29c4a1b-c07d-4849-b78b-3cca11edd539',
   '1f91b043-3937-4d78-b74e-c01486a0366a',
   '5b37a974-b6db-467f-81d5-0614bf259a8c',
   '1f1ed0de-ccc1-450c-86d2-74cf04664e70',
   '08f09ebd-8910-4092-bfee-f065e80f3c2a',
   '68764425-eb7b-4779-9548-c0e819f7b592',
   'efcb04b2-2094-42f1-b3f3-79b1b7ae8a36',
   '28b8af28-55ca-44dd-bbd1-bdc9057bfb5f']],
 'distances': [[1.2179785925348834,
   1.2820421072387942,
   1.283722624762129,
   1.3394140314613365,
   1.3476100072530883,
   1.3497488412458007,
   1.3501860928900984,
   1.3516339373651218,
   1.3527374372908103,
   1.3601365266493244]],
 'metadatas': [[{'source': 'examples/data/2112.01488v3.pdf'},
   {'source': 'examples/data/2112.01488v3.pdf'},
   {'source': 'examples/data/2112.01488v3.pdf'},
   {'source': 'examples/data/2112.01488v3.pdf'},
   {'source': 'examples/data/2112.01488v3.pdf'},
   {'source': 'examples/data/2112.01488v3.pdf'},
   {'sourc

In [38]:
results['metadatas'][0]

[{'source': 'examples/data/2112.01488v3.pdf'},
 {'source': 'examples/data/2112.01488v3.pdf'},
 {'source': 'examples/data/2112.01488v3.pdf'},
 {'source': 'examples/data/2112.01488v3.pdf'},
 {'source': 'examples/data/2112.01488v3.pdf'},
 {'source': 'examples/data/2112.01488v3.pdf'},
 {'source': 'examples/data/2112.01488v3.pdf'},
 {'source': 'examples/data/2112.01488v3.pdf'},
 {'source': 'examples/data/2112.01488v3.pdf'},
 {'source': 'examples/data/2112.01488v3.pdf'}]

In [29]:
results.keys()

dict_keys(['ids', 'distances', 'metadatas', 'embeddings', 'documents', 'uris', 'data'])

## Retriever

In [ ]:
collection.query()

In [43]:
type(collection)

chromadb.api.models.Collection.Collection

In [51]:
from typing import Optional, Callable, List
from langchain_core.documents import Document
from chromadb.api.client import Client
from chromadb.api.models.Collection import Collection

class SimpleRAGRetriever(BaseRetriever):
    vector_store_client: Client = None
    collection_name: str = None
    # doc_embedding_function = None # probably don't need
    query_embedding_function: Optional[Callable[[str], List[float]]] = None
    collection: Collection = None
    k: int = 10
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.collection = self.vector_store_client.get_or_create_collection(name=self.collection_name)

    def _get_relevant_documents(self, query, run_manager):
        embedded_query = self.query_embedding_function(query)
        results = self.collection.query(query_embeddings=embedded_query, n_results=self.k)
        docs = []
        for n in range(len(results['ids'])):
            doc = Document(
                page_content=results['documents'][0][n],
                metadata=results['metadatas'][0][n]
                )
            docs.append(doc)
        return docs

In [52]:
simple_rag_retriever = SimpleRAGRetriever(vector_store_client=client, collection_name='simple_rag', query_embedding_function=hf.embed_query)



In [53]:
simple_rag_retriever.invoke('What is the capital of France?')



[Document(page_content='Subtopics\n\nWriting\n\nSearch Forum\n\n497 2003\n\n277k\n\nESL, Linguistics, Worldbuilding\n\n1071 2000\n\n200k\n\nEnglish\n\nRecreation\n\nSearch Forum\n\n563 2002\n\n263k\n\nSci-Fi, RPGs, Photography\n\n924 2002\n\n167k\n\nGaming, Anime, Movies\n\nScience\n\nSearch Forum\n\n538 2013\n\n344k\n\nChemistry, Statistics, Academia\n\n617 2017\n\n1.694M\n\nMath, Physics, Biology\n\nTechnology\n\nSearch Forum\n\n916 2003\n\n1.276M\n\nWeb Apps, Ubuntu, SysAdmin\n\n596 2004\n\n639k\n\nApple, Android, UNIX, Security\n\nLifestyle\n\nSearch Forum\n\n417 2076\n\n269k\n\nDIY, Music, Bicycles, Car Maintenance\n\n661 2002\n\n119k\n\nCooking, Sports, Travel\n\nPooled\n\nSearch Forum\n\n2931 10097\n\n2.4M\n\nAll of the above\n\n3869 10025\n\n2.8M All of the above\n\nTable 1: Composition of LoTTE showing topics, question sets, and a sample of corresponding subtopics. Search Queries are taken from GooAQ, while Forum Queries are taken directly from the StackExchange archive. The p

## Chat pipe

In [2]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import SQLChatMessageHistory


In [428]:
ChatOpenAI(model='gpt-3.5-turbo')

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7f936f640680>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f936f6431d0>, openai_api_key=SecretStr('**********'), openai_proxy='')

In [447]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a funny assistant"),
        ("human", "{yapping}")
    ]
)

chain = prompt | ChatOpenAI(model='gpt-3.5-turbo')

rtrn_msg = chain.invoke({'yapping': 'what is the capital of France?'})
rtrn_msg

AIMessage(content='The capital of France is "F." Just kidding! It\'s Paris.', response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 23, 'total_tokens': 38}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-10024577-732c-45ab-9e4d-0efad8b050e3-0')

In [451]:
prompt

ChatPromptTemplate(input_variables=['yapping'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a funny assistant')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['yapping'], template='{yapping}'))])

In [498]:
class ConversationAgent(param.Parameterized):
    history_messages = param.List(per_instance=True)
    system_prompt = param.String(default='You are a funny assistant', per_instance=True)
    
    def send(self, message):
        prompt = ChatPromptTemplate.from_messages(
            [
                ("system", self.system_prompt),
                MessagesPlaceholder('history'),
                ("human", "{message}")
            ]
        )
        chain = prompt | ChatOpenAI(model='gpt-3.5-turbo')
        reply = chain.invoke({'history': self.history_messages, 'message': message})
        self.history_messages.extend([('human', message), ('assistant', reply.content)])
        return reply



In [499]:
conv_agent = ConversationAgent()

In [492]:
conv_agent

In [500]:
conv_agent.history_messages

[('human', 'What is the capital of France?'),
 ('assistant',
  AIMessage(content='The capital of France is Paris, where you can find the Eiffel Tower, delicious pastries, and fashionable Parisians!', response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 23, 'total_tokens': 49}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e3a38cce-cf56-4ec2-b28c-9fe808c1ec94-0'))]

In [497]:
conv_agent.send("what is your real name? Answer me as if you are committing the Butlerian Jihad.")

AIMessage(content='I am an AI assistant and I do not have a personal name. My purpose is to assist you with your tasks and provide information to the best of my abilities. If you have any questions or need help, feel free to ask.', response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 114, 'total_tokens': 161}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1ef908f8-46cd-4504-84cb-ae1232c520d6-0')

In [496]:
conv_agent.history_messages

[('human', "That wasn't very funny"),
 ('assistant',
  'I guess I need to work on my comedy routine! Let me try again: Why did the scarecrow win an award? Because he was outstanding in his field!'),
 ('human', 'much better'),
 ('assistant',
  "Phew, I'm glad you found that one funny! Let me know if you need more jokes or anything else.")]

In [8]:
class TestClass(param.Parameterized):
    test_param = param.String()

    def set_param(self, val):
        self.test_param = val


In [9]:
test_class = TestClass()
test_class.test_param = 'some string'

In [10]:
test_class.test_param

'some string'

In [6]:
test_class.param

Name,Value,Type,Range
name,'TestClass00117',String,nullable constant
test_param,['something'],List,"(0, None)"


In [11]:
test_class2 = TestClass()
test_class2.test_param

''

# UI Layer

In [ ]:
# Model layer
class Chat(param.Parameterized):
    conversation_history = param.ClassSelector(
       class_=ConversationHistory,
       doc="Message handler between AI and User - could contain source messages as well")
    context_history = param.ClassSelector(
        class_=ContextHandler,
        is_instance=True,
        doc="Context handler")
    retriever_store = param.ClassSelector(
        class_=RetrieverStore,
        is_instance=True,
        doc="Retriever store")
    retrieval = param.ClassSelector(
        class_=BaseRAG,
        is_instance=True,
        doc="RAG handler")
    reply = param.ClassSelector(
        class_=ReplyHandler,
        is_instance=True,
        doc="Last Generated Reply")
    # sources = param.List(default=[], doc="List of sources to retrieve")
    prompt_template = param.ClassSelector(
        class_=ChatPromptTemplate,
        is_instance=True,
        doc="Prompt template for the chat model")

    
    def generate_reply(self, message):
        """"""
        sources = self.rag.get_sources(message) 
        context = self.context_history.get_context(sources)
        prompt = self.prompt_template.format(context=context)
        return stream
        




In [24]:
# View Layer
class ChatView(Viewer):

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
        self.chat_input = pn.chat.ChatAreaInput()
        self.chat_feed = pn.chat.ChatFeed()
        self.chat_send_button = pn.widgets.Button(name='Send')
        self.chat_retrieve_switch = pn.widgets.Switch(name='Retrieve')
        self.corpus_selector = pn.widgets.Select(options=['simple_rag', 'custom_rag'])
        self.corpus_parameters
        
    def chat_input(self):
        return pn.chat.ChatAreaInput()
    
    def chat_feed(self):
        return pn.chat.ChatFeed()
    
    def __panel__(self):
        return pn.Column(self.chat_feed(), self.chat_input())

In [ ]:
# Controller Layer
class ChatController(param.Parameterized):
    view = param.ClassSelector(class_=ChatView, is_instance=True)
    model = param.ClassSelector(class_=BaseLanguageModel, is_instance=True)

    def __panel__(self):
        return self.view()

In [26]:
pn.panel(ChatInterface())

BokehModel(combine_events=True, render_bundle={'docs_json': {'c0cc8570-7eb7-4dcb-9412-cd85afe5b670': {'version…

In [45]:
def on_click(event):
    print("ChatMessage clicked!")

message = pn.chat.ChatMessage(pn.Row(pn.widgets.Button(name='click me', height=50), pn.pane.Markdown('# some test'), pn.widgets.Button(name='click me', height=50),  pn.pane.Markdown('# some test')))
serve(message)

Launching server at http://localhost:35594


In [50]:
chat_feed.send('interior crocadile alligator')

BokehModel(combine_events=True, render_bundle={'docs_json': {'682c0bce-e79a-4b7f-9753-e8290db425be': {'version…

In [48]:
import asyncio

In [65]:
chat_feed = pn.chat.ChatFeed()

# creates a new message
message = chat_feed.stream("Hello", user="Aspiring User", avatar="🤓")
chat_feed

BokehModel(combine_events=True, render_bundle={'docs_json': {'e465d4fe-140f-4671-8dac-d50f2ef73d36': {'version…

In [67]:
# streams (appends) to the previous message
message = chat_feed.stream(
    'what is the capital of France?',
    user="reeeee",
    avatar="🤓",
    message=message,
)

In [64]:
# streams (appends) to the previous message
message = chat_feed.stream(
    pn.pane.Markdown('is it me'),
    user="reeeee",
    avatar="🤓",
    message=message,
)